# 一橋大学のニュースをオブジェクト保存

ここまで何度も行ってきた、一橋大学のニューステキストを辞書型(dict型)に変換する作業を、Pythonのオブジェクトとして保存する。
保存したものを、復元して使えるようにします。

標準ライブラリ　``pickle`` を使うと、Pythonのオブジェクトをそのままファイルに保存することができます。
機械学習の学習済みモデルのように生成するのに時間がかかったり、何度も使うことが有るものに使うことができます。

基本的なPythonのデータ型であればpickleして保存することができます。（一部できないものもあります）

このような考え方にはJSON形式にして保存する方法もありますが、データ型が維持できない点で、機械学習の分野では、pickleが一般的に使われています。


## 辞書の生成

今までどおり、ニュースのテキストから、形態素解析を行い、辞書型のオブジェクトを作る

In [1]:
import urllib.request
import base64

out = []
text = ""

url = "https://hit-u-data-text-processing.herokuapp.com/data/20210502-news-text.txt"
auth_str = base64.b64encode(b"reader:hit-u")
req = urllib.request.Request(url,
                            headers={"Authorization": "Basic " + auth_str.decode("utf-8")})
with urllib.request.urlopen(req) as req:
    text = req.read().decode("utf-8")
    for line in text.split("\n"):
        if line.strip() == "===":
            if text:
                out.append(text)
            text = ""
        else:
            text += line
url = "https://hit-u-data-text-processing.herokuapp.com/data/20210612-news-text.txt"
auth_str = base64.b64encode(b"reader:hit-u")
req = urllib.request.Request(url,
                            headers={"Authorization": "Basic " + auth_str.decode("utf-8")})
with urllib.request.urlopen(req) as req:
    text = req.read().decode("utf-8")
    for line in text.split("\n"):
        if line.strip() == "===":
            if text:
                out.append(text)
            text = ""
        else:
            text += line

In [2]:
# !pip install janome

In [3]:
from janome.tokenizer import Tokenizer
from janome.tokenfilter import TokenFilter
from janome.tokenfilter import CompoundNounFilter
from janome.tokenfilter import POSKeepFilter
from janome.tokenfilter import LowerCaseFilter
from janome.charfilter import UnicodeNormalizeCharFilter
from janome.analyzer import Analyzer

In [4]:
class StopWordFilter(TokenFilter):
    def __init__(self, words):
        self.stop_words = words
    
    def apply(self, tokens):
        for token in tokens:
            if token.surface not in self.stop_words:
                yield token

In [5]:
import urllib.request
import base64
url = "https://hit-u-data-text-processing.herokuapp.com/data/stopwords.txt"
auth_str = base64.b64encode(b"reader:hit-u")
req = urllib.request.Request(url,
                            headers={"Authorization": "Basic " + auth_str.decode("utf-8")})

stop_words = []
with urllib.request.urlopen(req) as req:
    lines = req.read().decode("utf-8")
    for line in lines.split("\n"):
        if line.strip():
            stop_words.append(line.strip())
stop_words

['大学', '一橋大学', '===', 'こと', 'the', 'ため', 'よう', 'of', '(', ')', '様']

In [6]:
stop_word_filter = StopWordFilter(stop_words)

In [7]:
token_filters = [CompoundNounFilter(),
                POSKeepFilter(["名詞", "動詞", "形容詞"]),
                LowerCaseFilter(),
                stop_word_filter]

In [8]:
char_filters = [UnicodeNormalizeCharFilter()]
tokenizer = Tokenizer()
analyzer = Analyzer(char_filters=char_filters, 
                    tokenizer=tokenizer, 
                    token_filters=token_filters)

In [9]:
docs_words = {}
for i, text in enumerate(out):
    docs_words[i] = []
    for token in analyzer.analyze(text):
        docs_words[i].append(token.base_form)

## pickleで永続化

Pythonのオブジェクトをシリアライズ化するpickeを使って、ファイルに保存（永続化)を行います。

In [10]:
import pickle

In [11]:
with open("docs_words.picke", "wb") as f:
    pickle.dump(docs_words, f)

docs_words.picke というファイルが出来上がりました。

## pickeデータの復元

データを復元してみます。

In [12]:
with open("docs_words.picke", "rb") as f:
    docs_words_new = pickle.load(f)

In [13]:
type(docs_words_new)

dict

In [14]:
docs_words_new[0][:10]

['ソニー',
 'パナソニック',
 '富士通',
 '資生堂',
 '共同',
 'デザイン組織',
 '共通評価指標',
 '検討',
 '作成',
 '2021年4月30日']

In [15]:
docs_words is docs_words_new

False

In [16]:
docs_words == docs_words_new

True